In [1]:
import keras
import pandas as pd
import numpy as np
import random
import sklearn
# multivariate output stacked lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import LSTM,Dense
from sklearn.preprocessing import MinMaxScaler


Using TensorFlow backend.


In [2]:
df = pd.read_csv("daily_coinbase.csv")

dataset=df[["coinbase"]].fillna(method="ffill")
scl = MinMaxScaler(feature_range = (0, 1))
#Scale the data
dataset = scl.fit_transform(dataset)
dataset = dataset.squeeze().tolist()

n_steps_in = 14
n_steps_out = 1
def processData(data,lb):
    X,Y = [],[]
    for i in range(len(data)-lb-1):
        X.append(data[i:(i+lb)])
        Y.append(data[(i+lb)])
    return np.array(X).reshape((len(X), n_steps_in, 1)),np.array(Y)

X,y = processData(dataset,n_steps_in)

'''
# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    
    return np.array(X).reshape((len(X), n_steps_in, 1)), np.array(y)
 
# define input sequence
#random.shuffle(dataset)
# choose a number of time steps
n_steps_in, n_steps_out = 10, 1
# split into samples
X, y = split_sequence(dataset, n_steps_in, n_steps_out)
'''

'\n# split a univariate sequence into samples\ndef split_sequence(sequence, n_steps_in, n_steps_out):\n    X, y = list(), list()\n    for i in range(len(sequence)):\n        # find the end of this pattern\n        end_ix = i + n_steps_in\n        out_end_ix = end_ix + n_steps_out\n        # check if we are beyond the sequence\n        if out_end_ix > len(sequence):\n            break\n        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]\n        X.append(seq_x)\n        y.append(seq_y)\n    \n    return np.array(X).reshape((len(X), n_steps_in, 1)), np.array(y)\n \n# define input sequence\n#random.shuffle(dataset)\n# choose a number of time steps\nn_steps_in, n_steps_out = 10, 1\n# split into samples\nX, y = split_sequence(dataset, n_steps_in, n_steps_out)\n'

In [3]:
dataset

[0.09747555883591161,
 0.12270944482262003,
 0.16845289851208126,
 0.17033520730930615,
 0.18950539606793287,
 0.14428938235403574,
 0.2908494207588483,
 0.17533418899353317,
 0.0857788297094221,
 0.0673483880008828,
 0.05278916142740387,
 0.15020417123317095,
 0.09379026384699515,
 0.07022501731369249,
 0.13007645469855256,
 0.0678046483609406,
 0.04932989454851367,
 0.08091135080127002,
 0.07144391879772344,
 0.05790702669065655,
 0.047024958182076464,
 0.05141858737157166,
 0.16796992234682045,
 0.05669977934400843,
 0.08479240777774777,
 0.12613150176345755,
 0.09928981213161017,
 0.06805931390305162,
 0.11840930830015575,
 0.14767054267036586,
 0.1828544290500953,
 0.16411065197715732,
 0.26642261855686666,
 0.1412281867785196,
 0.15997198644336266,
 0.1618022001496307,
 0.07361287959380422,
 0.053011444924680164,
 0.09581027704813605,
 0.17278208650191026,
 0.08820444191540175,
 0.0708213083235068,
 0.0783575403047582,
 0.03422098362768088,
 0.041376417963950536,
 0.0709496286798

In [3]:
# split into train and test
n_train = int(len(X) - 7)
trainX, testX = X[:n_train, :], X[n_train:, :]
trainy, testy = y[:n_train], y[n_train:]

In [4]:
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (trainX.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 30))
regressor.add(Dropout(0.3))
regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics=['mse'])

'''
es = EarlyStopping(monitor='val_loss',
                                  min_delta=0,
                                  patience=40,
                                  verbose=0, mode='auto')
'''

history = regressor.fit(X, y, epochs =500, batch_size = 64, verbose=2)

W0715 01:14:19.403719  7888 deprecation_wrapper.py:119] From C:\Users\ryan_\.conda\envs\forecasting_tf\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0715 01:14:19.419718  7888 deprecation_wrapper.py:119] From C:\Users\ryan_\.conda\envs\forecasting_tf\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0715 01:14:19.422712  7888 deprecation_wrapper.py:119] From C:\Users\ryan_\.conda\envs\forecasting_tf\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0715 01:14:19.638643  7888 deprecation_wrapper.py:119] From C:\Users\ryan_\.conda\envs\forecasting_tf\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_w

Epoch 1/500
 - 3s - loss: 0.0164 - mean_squared_error: 0.0164
Epoch 2/500
 - 0s - loss: 0.0116 - mean_squared_error: 0.0116
Epoch 3/500
 - 0s - loss: 0.0113 - mean_squared_error: 0.0113
Epoch 4/500
 - 0s - loss: 0.0114 - mean_squared_error: 0.0114
Epoch 5/500
 - 0s - loss: 0.0108 - mean_squared_error: 0.0108
Epoch 6/500
 - 0s - loss: 0.0110 - mean_squared_error: 0.0110
Epoch 7/500
 - 0s - loss: 0.0111 - mean_squared_error: 0.0111
Epoch 8/500
 - 0s - loss: 0.0109 - mean_squared_error: 0.0109
Epoch 9/500
 - 0s - loss: 0.0105 - mean_squared_error: 0.0105
Epoch 10/500
 - 0s - loss: 0.0111 - mean_squared_error: 0.0111
Epoch 11/500
 - 0s - loss: 0.0106 - mean_squared_error: 0.0106
Epoch 12/500
 - 0s - loss: 0.0105 - mean_squared_error: 0.0105
Epoch 13/500
 - 0s - loss: 0.0106 - mean_squared_error: 0.0106
Epoch 14/500
 - 0s - loss: 0.0107 - mean_squared_error: 0.0107
Epoch 15/500
 - 0s - loss: 0.0101 - mean_squared_error: 0.0101
Epoch 16/500
 - 0s - loss: 0.0107 - mean_squared_error: 0.0107
E

Epoch 131/500
 - 0s - loss: 0.0069 - mean_squared_error: 0.0069
Epoch 132/500
 - 0s - loss: 0.0068 - mean_squared_error: 0.0068
Epoch 133/500
 - 0s - loss: 0.0074 - mean_squared_error: 0.0074
Epoch 134/500
 - 0s - loss: 0.0074 - mean_squared_error: 0.0074
Epoch 135/500
 - 0s - loss: 0.0070 - mean_squared_error: 0.0070
Epoch 136/500
 - 0s - loss: 0.0071 - mean_squared_error: 0.0071
Epoch 137/500
 - 0s - loss: 0.0074 - mean_squared_error: 0.0074
Epoch 138/500
 - 0s - loss: 0.0072 - mean_squared_error: 0.0072
Epoch 139/500
 - 0s - loss: 0.0073 - mean_squared_error: 0.0073
Epoch 140/500
 - 0s - loss: 0.0073 - mean_squared_error: 0.0073
Epoch 141/500
 - 0s - loss: 0.0071 - mean_squared_error: 0.0071
Epoch 142/500
 - 0s - loss: 0.0070 - mean_squared_error: 0.0070
Epoch 143/500
 - 0s - loss: 0.0072 - mean_squared_error: 0.0072
Epoch 144/500
 - 0s - loss: 0.0072 - mean_squared_error: 0.0072
Epoch 145/500
 - 0s - loss: 0.0070 - mean_squared_error: 0.0070
Epoch 146/500
 - 0s - loss: 0.0072 - mea

 - 0s - loss: 0.0068 - mean_squared_error: 0.0068
Epoch 260/500
 - 0s - loss: 0.0068 - mean_squared_error: 0.0068
Epoch 261/500
 - 0s - loss: 0.0068 - mean_squared_error: 0.0068
Epoch 262/500
 - 0s - loss: 0.0067 - mean_squared_error: 0.0067
Epoch 263/500
 - 0s - loss: 0.0070 - mean_squared_error: 0.0070
Epoch 264/500
 - 0s - loss: 0.0066 - mean_squared_error: 0.0066
Epoch 265/500
 - 0s - loss: 0.0070 - mean_squared_error: 0.0070
Epoch 266/500
 - 0s - loss: 0.0065 - mean_squared_error: 0.0065
Epoch 267/500
 - 0s - loss: 0.0066 - mean_squared_error: 0.0066
Epoch 268/500
 - 0s - loss: 0.0068 - mean_squared_error: 0.0068
Epoch 269/500
 - 0s - loss: 0.0065 - mean_squared_error: 0.0065
Epoch 270/500
 - 0s - loss: 0.0064 - mean_squared_error: 0.0064
Epoch 271/500
 - 0s - loss: 0.0067 - mean_squared_error: 0.0067
Epoch 272/500
 - 0s - loss: 0.0066 - mean_squared_error: 0.0066
Epoch 273/500
 - 0s - loss: 0.0066 - mean_squared_error: 0.0066
Epoch 274/500
 - 0s - loss: 0.0064 - mean_squared_erro

Epoch 388/500
 - 0s - loss: 0.0063 - mean_squared_error: 0.0063
Epoch 389/500
 - 0s - loss: 0.0059 - mean_squared_error: 0.0059
Epoch 390/500
 - 0s - loss: 0.0061 - mean_squared_error: 0.0061
Epoch 391/500
 - 0s - loss: 0.0057 - mean_squared_error: 0.0057
Epoch 392/500
 - 0s - loss: 0.0056 - mean_squared_error: 0.0056
Epoch 393/500
 - 0s - loss: 0.0056 - mean_squared_error: 0.0056
Epoch 394/500
 - 0s - loss: 0.0054 - mean_squared_error: 0.0054
Epoch 395/500
 - 0s - loss: 0.0054 - mean_squared_error: 0.0054
Epoch 396/500
 - 0s - loss: 0.0052 - mean_squared_error: 0.0052
Epoch 397/500
 - 0s - loss: 0.0056 - mean_squared_error: 0.0056
Epoch 398/500
 - 0s - loss: 0.0054 - mean_squared_error: 0.0054
Epoch 399/500
 - 0s - loss: 0.0054 - mean_squared_error: 0.0054
Epoch 400/500
 - 0s - loss: 0.0058 - mean_squared_error: 0.0058
Epoch 401/500
 - 0s - loss: 0.0057 - mean_squared_error: 0.0057
Epoch 402/500
 - 0s - loss: 0.0052 - mean_squared_error: 0.0052
Epoch 403/500
 - 0s - loss: 0.0053 - mea

In [6]:
regressor.save("forecasting_model.h5")

In [ ]:
tfjs.converters.save_keras_model(regressor, "forecast_model")

In [ ]:
testy

In [ ]:
pd.DataFrame([predicted, testy])

In [ ]:
plt.plot(history.history["loss"])
plt.show()

In [ ]:
predicted=regressor.predict(X[-30:])
plt.figure(figsize=(20,5))
plt.plot(predicted.squeeze(), label="predicted")
plt.plot(y[-30:].squeeze(), label="actual")
plt.legend()
plt.show()

In [ ]:
x_input = X[-1]
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

In [ ]:
test_=df.Avg_Max_Temp.values.tolist()[-44:]

In [ ]:
np.array(test[:30])

In [ ]:
test=np.array(test_[:30]).reshape((1, 30, 1))

In [ ]:
test_y = model.predict(test, verbose=1)

In [ ]:
plt.plot(list(test_y[0]), label = "predicted")
plt.plot(test_[30:], label='true_values')
plt.legend()
plt.show()

In [ ]:
import tensorflow